### Lab Assignment: Commercial Data Analysis

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023


#### Jiaxing Qiu
#### JQ2UW
---

### INSTRUCTIONS  
In this assignment, you will work with a dataset containing information about businesses.  
Each record is a business location.  Follow the steps below, writing and running the code in blocks, and displaying the solutions.  

Each question part is worth 1 POINT, for a total of 15 POINTS.

Hint: reaching deeper fields in json hierarchy can be done like this:  

`df.select('address.street_number')`

---

In [1]:
# pip install pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

In [1]:
# note that read.json can read a zipped JSON directly

**1. (1 PT) Read in the dataset and show the number of records**

In [21]:
df = spark.read.json('/project/ds5559/large_datasets/part-00000-a159c41a-bc58-4476-9b78-c437667f9c2b-c000.json.gz')

In [5]:
df.count()

154679

**2. (1 PT) Show the first 5 records**

In [6]:
df.show(5)

+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|             address|       business_tags|               hours|              id|menu|             reviews|                urls|             webpage|
+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|{Woodburn, {45.15...|                null|                null|000023995a540868|null|                  []|{woodburn.k12.or....|{Educational Tech...|
|{Hialeah, {25.884...|{[], [{has_atm, Y...|{null, 1900, null...|0000821a1394916e|null|                null|{null, [yelp.com]...|                null|
|{Rochester, {43.1...|{[], [{accepts_cr...|{null, 1700, null...|000136e65d50c3b7|null|[{New (to me) qui...|{usps.com, [yelp....|{Welcome | USPS G...|
|{West Palm Beach,...|                null|                null|00014329a70b9869|null|              

**3. (1 PT) Show the first 5 street addresses which are not null**  

In [9]:
df.filter(df.address.isNotNull()).show(5)

+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|             address|       business_tags|               hours|              id|menu|             reviews|                urls|             webpage|
+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|{Woodburn, {45.15...|                null|                null|000023995a540868|null|                  []|{woodburn.k12.or....|{Educational Tech...|
|{Hialeah, {25.884...|{[], [{has_atm, Y...|{null, 1900, null...|0000821a1394916e|null|                null|{null, [yelp.com]...|                null|
|{Rochester, {43.1...|{[], [{accepts_cr...|{null, 1700, null...|000136e65d50c3b7|null|[{New (to me) qui...|{usps.com, [yelp....|{Welcome | USPS G...|
|{West Palm Beach,...|                null|                null|00014329a70b9869|null|              

**4. (1 PT) Location**  

Count the number of records where the city is Phoenix

In [10]:
from pyspark.sql.functions import col, length

df.filter(col("address.city") == "Phoenix").count()

762

**5. (1 PT) Hours**  

Count the number of records where closing time on Thursday is 8pm

In [11]:
df.filter(col("hours.thursday_close") == "2000").count()

3313

**6. (1 PT) Location and Hours**  

Count the number of records where city is Phoenix and closing time on Thursday is 8pm

In [12]:
df.filter((col("address.city") == "Phoenix") & (col("hours.thursday_close") == "2000")).count()

12

**7. (1 PT) Price Range**  

Price range is quoted in number of dollar signs.  Count the number of records with price range greater than or equal to two.

In [13]:
df.filter((col("menu.price_range")) >= 2).count()

1135

**8. (1 PT) COMPANY HEADQUARTERS**  

For the `address.is_headquarters` field:  
how many locations are HQ / are NOT HQ / are null?

In [14]:
print("Count of HQ:")
df.filter(col("address.is_headquarters") == True).count()

Count of HQ:


318

In [ ]:
# another way
df.filter(col("address.is_headquarters")).count()

In [15]:
print("Count of NOT HQ:")
df.filter(col("address.is_headquarters") == False).count()

Count of NOT HQ:


66736

In [19]:
# another way
df.filter(~col("address.is_headquarters")).count()

66736

In [16]:
print("Count of NULL:")
df.filter(col("address.is_headquarters").isNull()).count()

Count of NULL:


87625

**9. (1 PT) Webpage URLs**  

Register the dataframe as a temp table.  
Next, use Spark SQL to select only the webpage title column, filtering on rows where the webpage url (accessed under `webpage.url`) is *Target.com*. 

Show only one resulting row and don't truncate the output.

In [23]:
df.createOrReplaceTempView("bsns_temp")

In [24]:
spark = SparkSession.builder.getOrCreate()
query = "SELECT webpage.title FROM bsns_temp WHERE webpage.url = 'Target.com' LIMIT 1"
result = spark.sql(query)
result.show(truncate=False)

+-------------------------------+
|title                          |
+-------------------------------+
|Target : Expect More. Pay Less.|
+-------------------------------+



**10. (1 PT) Analysis on Ratings**  

The reviews contains information such as the number of stars for each review (the *rating*).  
The ratings are stored in an array (`reviews.stars`) for each business location (you should check for yourself). Return the top five most common rating arrays.  For example, an array might look like: 
[5, 5]



In [30]:
from pyspark.sql.functions import col, array_sort, desc
df.groupBy("reviews.stars").count().sort(desc("count")).limit(5).show(truncate=False)

+------+-----+
|stars |count|
+------+-----+
|null  |74679|
|[]    |42419|
|[5]   |4258 |
|[null]|3067 |
|[5, 5]|1610 |
+------+-----+



**11. More work with Ratings**  

For this question, you will filter out null ratings and then compute the average rating for each business location (using the field: `id`).


a) (1 PT) Create a new dataframe retaining two fields: `id`, `reviews.stars`


In [31]:
new_df = df.filter(col("reviews.stars").isNotNull()).select("id", "reviews.stars")
new_df.show()

+----------------+--------------------+
|              id|               stars|
+----------------+--------------------+
|000023995a540868|                  []|
|000136e65d50c3b7|              [4, 4]|
|0003b7589a4e12a0|                 [5]|
|00045f958e4bb02a|[null, null, null...|
|00059519f0dba1b4|[null, null, null...|
|0006d5aa170bae22|                  []|
|0008bc70f8ba62bf|              [null]|
|000a1df4c8e0ecd2|[null, null, 4, 5...|
|000bf1e934ac9cb6|                  []|
|000c4037ef6d4b3b|                  []|
|000c7b7a30623083|                 [5]|
|000c9ffc8b89af03|[5, 2, 5, 3, 3, 1...|
|000ca67c3cf252e5|                  []|
|000de20baa847ecc|  [1, 1, 1, 1, 5, 1]|
|000e439e7667839d|                  []|
|001064359d9f162f|     [5, 5, 5, 5, 5]|
|0010c9f495d87dd7|[5, 1, 1, 5, 3, 5...|
|0012eac5aaf0bd45|                  []|
|0013cd52c783f818|                  []|
|0017774db5e6400a|[null, 5, 5, 5, 5...|
+----------------+--------------------+
only showing top 20 rows



b) (1 PT) Create a row for each rating  
hint: use the `withColumn()` and `explode()` functions  
you will need to import the `explode()` function by issuing:

`from pyspark.sql.functions import explode`


In [38]:
# explode function 
# id1 has a [5,5,4,3,2] -> column[id,rating;1,5;1,5;] # same to R melt
from pyspark.sql.functions import explode

explode_df = df.withColumn("rating", explode("reviews.stars"))
explode_df.select("id","rating").show()

+----------------+------+
|              id|rating|
+----------------+------+
|000136e65d50c3b7|     4|
|000136e65d50c3b7|     4|
|0003b7589a4e12a0|     5|
|00045f958e4bb02a|  null|
|00045f958e4bb02a|  null|
|00045f958e4bb02a|  null|
|00045f958e4bb02a|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|  null|
|00059519f0dba1b4|     1|
|00059519f0dba1b4|     5|
|00059519f0dba1b4|     2|
|00059519f0dba1b4|     4|
|00059519f0dba1b4|     5|
+----------------+------+
only showing top 20 rows



c) (1 PT) Return a count of the number of ratings in this dataframe

In [39]:
explode_df.select("rating").count()

600082

d) (1 PT) Drop rows where the rating is null, and return a count of the number of non-null ratings

In [40]:
explode_df.select("rating").filter(col("rating").isNotNull()).count()

538241

e) (1 PT) Compute the average rating, grouped by `id`. After the average is computed, sort by `id` in ascending order and show the top 10 records.  
 
hint:   
this can all be done in one line using the `agg()` function  
this `id` should be at the top: 000136e65d50c3b7

In [44]:
from pyspark.sql.functions import avg

explode_df.groupBy("id").agg(avg("rating").alias("average_rating")).orderBy("id").show(10)

+----------------+------------------+
|              id|    average_rating|
+----------------+------------------+
|000136e65d50c3b7|               4.0|
|0003b7589a4e12a0|               5.0|
|00045f958e4bb02a|              null|
|00059519f0dba1b4|3.3333333333333335|
|0008bc70f8ba62bf|              null|
|000a1df4c8e0ecd2|               4.6|
|000c7b7a30623083|               5.0|
|000c9ffc8b89af03|               3.0|
|000de20baa847ecc|1.6666666666666667|
|001064359d9f162f|               5.0|
+----------------+------------------+
only showing top 10 rows



In [46]:
df.describe("id").show()

+-------+----------------+
|summary|              id|
+-------+----------------+
|  count|          154679|
|   mean|        Infinity|
| stddev|             NaN|
|    min|000023995a540868|
|    max|fffed87e613292b7|
+-------+----------------+

